# MS 141 Lecture 5

# Numerical Differentiation
## Reading materials: second half of Chapter 5 of Newman's book.<br>

Numerical differentiation is the workhorse of numerical methods in scientific computing.<br> 
We will examine simple approaches for differentiating a function of one variable.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Set common figure parameters
newparams = {'figure.figsize': (10, 5), 'axes.grid': True,
             'lines.linewidth': 1.5, 'lines.markersize': 10,
             'font.size': 14}
plt.rcParams.update(newparams)

## 1. Derivative
The derivative of a function $f(x)$ at $x$ is the limit

$$ f'(x) = \lim_{h \to 0} \frac{f(x+h) - f(x)}{h}$$

The derivative can be calculated analytically when $f(x)$ is a simple continuous function. 
For example, if $f(x) = x^3$, then $f'(x) = 3 x^2$, and no numerical calculation is needed. 

Often in numerical calculations one does not have $f(x)$ in analytic form. Rather,  $f(x)$ could be a data set defined on a grid of points $x_n$, which may or may not be regularly spaced. The question of how to compute the derivative of this data set requires further thought.<br> 

In addition, the definition above is not suitable for numerical calculations $-$ in the limit of vanishingly small $h$, the denominator causes an underflow and is approximated to zero, and the numerator has a large round off error. The limit is not well defined on a computer!<br>

The strategy for numerical methods to find approximate derivatives is to compute **differences** between the data at the various points, finding methods that are accurate up to some power of $h$, so that a small but finite value of $h$ can be used. 

## 2. Difference formulas

The three main approaches for computing derivatives are the **forward, backward, and central difference** formulas.<br> 
We will introduce them here and compare their error and accuracy in the next section.

The forward difference formula with step size $h$ is
$$ f'(x) \approx \frac{f(x+h) - f(x)}{h} $$

The backward difference formula with step size $h$ is
$$ f'(x) \approx \frac{f(x) - f(x-h)}{h} $$

The central difference formula with step size $h$ is
$$ f'(x) \approx \frac{f(x+h) - f(x-h)}{2h} $$
It is the average of the forward and backward difference formulas.

Note that in all these formulas, the limit $h \to 0$ is gone, and now we have a finite value of the step or grid size $h$. The points $x_n$ are arranged on a grid, and the function $f(x_n)$ is an array. The three approximations are shown schematically below.

In [ ]:
x = np.linspace(0,1,100)
h = 0.15

def f(x):
    return x**4 - x + 1

plt.figure(figsize=(10,5))
plt.plot(x,f(x),'k-')

#add lines
plt.plot([0.6-h, 0.6], [f(0.6-h),f(0.6)], 'b-', lw=2)
plt.plot([0.6, 0.6+h], [f(0.6),f(0.6+h)], 'r-', lw=2)
plt.plot([0.6-h, 0.6+h], [f(0.6-h)-0.05,f(0.6+h)-0.05], 'g-', lw=2)

#add points
plt.plot(0.6, f(0.6), 'go', markersize=12) 
plt.plot(0.6+h, f(0.6+h), 'ro', markersize=12)
plt.plot(0.6-h, f(0.6-h), 'bo', markersize=12)


plt.xlim(0.2,1.0)
plt.ylim(0.5,0.8)
plt.ylabel('$f(x)$',fontsize=16)

plt.legend(('f(x)','Backward difference', 'Forward difference', 'Central difference'),
           loc='upper center',fontsize=16)

plt.show()

These definitions can be extended to multiple dimensions, where the derivatives become gradients.<br> 
The numerical derivatives defined above form the basis for so-called *finite difference* methods to solve differential equations.

## 3. Implementation 

The three difference formulas are easy to implement. Let's write a function called `derivative` which takes as input parameters the function `f`, the point `a` at which we wish to compute the derivative, the `method` to compute the derivative, and the increment `h`. We set the default values `method='central'` and `h=0.01`. The routine returns $f'(x=a)$ computed with the corresponding difference formula with step size $h$.

In [ ]:
def derivative(f,a,method='central',h=0.01):
    '''Compute the difference formula for f'(x) with step size h.

    Parameters
    ----------
    f : function
        Array function of one variable
    a : number
        Compute derivative at x = a
    method : string
        Difference formula: 'forward', 'backward' or 'central'
    h : number
        Step size in difference formula. The default is h=0.01

    Returns
    -------
    float
        Difference formula at x=a:
            central: f(a+h) - f(a-h))/2h
            forward: f(a+h) - f(a))/h
            backward: f(a) - f(a-h))/h            
    '''
    if method == 'central':
        return (f(a + h) - f(a - h))/(2*h)
    elif method == 'forward':
        return (f(a + h) - f(a))/h
    elif method == 'backward':
        return (f(a) - f(a - h))/h
    else:
        raise ValueError("Method must be 'central', 'forward' or 'backward'.")

### Examples

Using   $f(x) = x^3$, we know that $f'(x) = 3 x^2$. Let's compute $f'(1)$, for which we expect a result of 3.

In [ ]:
f = lambda x: x**3
a = 1.0
diff_c = derivative(f,a,'central',0.01)
print(diff_c) 
# we set h = 0.01; note the error of order h^2 in the result

The central derivative works well. Let's try the forward and backward.

In [ ]:
diff_f = derivative(f,a,'forward',0.01)
diff_b = derivative(f,a,'backward',0.01)
print (diff_f)
print (diff_b)
# we set h = 0.01; note the error of order h in the result!

It's clear that the error is greater with the forward and backward derivatives. Let's carry out a more thorough comparison.<br> 
We know that the derivative of $\sin(x)$ is the function $\cos(x)$.

In [ ]:
# Define the grid as 100 points between 0 and 2pi
x = np.linspace(0,2*np.pi,100)
diff_c = derivative(np.sin,x,'central',0.02) #try also h=0.2
diff_f = derivative(np.sin,x, 'forward',0.02) #try also h=0.2

# the exact derivative is cos(x)
exact = np.cos(x)

plt.figure(figsize=(10,5))
plt.plot(x,diff_c,'g.',label='Central difference')
plt.plot(x,diff_f,'r.',label='Forward difference')
plt.plot(x,exact,'b',label='Analytical derivative')

plt.title('Derivative of the function sin(x)',fontsize=14)
plt.legend(loc='best',fontsize=14)
plt.show()

We see that for a small enough step size $h$, both methods work reasonably well. For larger values of $h$, the higher accuracy of the central difference method becomes apparent.

### Differentiation with NumPy and SciPy
Differentiation can be done easily with NumPy or Scipy.<br>
The SciPy function `scipy.misc.derivative` computes derivatives using the central difference formula.

In [ ]:
# x^3 function with scipy

from scipy.misc import derivative

'''
parameters in scipy.misc.derivative:
scipy.misc.derivative(func, x0, dx=1.0, n=1, args=(), order=3)
func : function
Input function.

x0 : float
The point at which the n-th derivative is computed.
Can be an array, in which case the entire derivative function is returned

dx : float, optional
Step size / increment (above, we called it h).

n : int, optional
Order of the derivative. Default is 1.

args : tuple, optional
Arguments

order : int, optional
Number of points to use, must be odd.
'''

f = lambda x: x**3
diff=derivative(f, 1.0, dx=1e-2, n=1,args=(),order=3)
print(diff)

The NumPy function `gradient` computes the derivative of an entire array, which represents a function over an interval,<br> 
using a central difference formula in the interior of the domain and a backward / forward formula at the boundaries.

In [ ]:
# sine function differentiation with numpy
x = np.linspace(0,2*np.pi,100)
f = np.sin(x)
diff = np.gradient(f,x)

#plot exact for comparison
exact = np.cos(x)

plt.plot(x,diff,'ro', ms=4, label='numpy')
plt.plot(x,exact,'b-',label='exact')

plt.title('Derivative of the function sin(x)',fontsize=14)
plt.legend(loc='best',fontsize=14)
plt.show()

## 4. Error

The error is of key importance when evaluating derivatives. The error analysis uses the Taylor expansion of a function $f(x)$ at a small distance $h$ from a point $x$:

$$ f(x+h) \approx f(x) + \frac{df(x)}{dx}h + \frac{1}{2!}\frac{d^2f(x)}{dx^2}h^2 + \frac{1}{3!}\frac{d^3f(x)}{dx^3}h^3 + \,\,...$$

This means that the finite difference we take in the forward-derivative differentiation can be approximated as:

$$ \frac{f(x+h) - f(x)}{h} \approx \frac{df(x)}{dx} + \frac{h}{2!}\frac{d^2f(x)}{dx^2} + \frac{h^2}{3!}\frac{d^3f(x)}{dx^3} +\,\, ...$$

In the forward difference method, we use for the derivative

$$  \frac{df(x)}{dx} \biggr\rvert_{\rm fd} = \frac{f(x+h) - f(x)}{h} = \frac{df(x)}{dx} + \frac{h}{2!}\frac{d^2f(x)}{dx^2} + \,\,... = \frac{df(x)}{dx} + \mathcal{O}(h)$$

where the notation $\mathcal{O}(h)$ means "a term of order $h$".

We say that the approximation error in the forward derivative is of order $h$, where $h$ is the step size / grid spacing, or that the forward derivative is "first-order accurate". The error is larger at points where the curvature $f''(x)$ is larger, and it vanishes if the curvature is zero since the function is a straight line in that case, so the forward / backward differences make no error.

Since the error is of order $h$, we can make the error smaller by making the grid size $h$ smaller, but if $h$ is too small one can lose precision due to round-off errors (see the Lecture on numerical errors). One also needs to evaluate the derivative at more points as the grid size gets smaller, so overall the cost of decreasing the grid spacing is in general significant. The backward difference case is fully analogous. 

Central differences, as mentioned above, are more accurate. We'll show that the central difference approach has an error of order $h^2$ (it is second-order accurate), so one can use a larger value of $h$ than in the forward difference case, or for a given value of $h$, obtain more accurate results. The error for the central-difference scheme is computed using
$$ f(x + h) = f(x) + f'(x)h + \frac{f''(x)}{2}h^2 + \frac{f'''(x)}{6}h^3 + \,\,... $$

$$ f(x - h) = f(x) - f'(x)h + \frac{f''(x)}{2}h^2 - \frac{f'''(x)}{6}h^3 + \,\,... $$
so that
$$ \frac{f(x+h) - f(x-h)}{2h} = f'(x) + \frac{f'''(x)}{6}h^2 + \mathcal{O}(h^4)$$

In the central difference method, we use for the derivative

$$  \frac{df(x)}{dx} \biggr\rvert_{\rm cd} = \frac{f(x+h) - f(x-h)}{2h} = \frac{df(x)}{dx} + \frac{f'''(x)}{6}h^2 + \mathcal{O}(h^4)$$

so that the leading term in the error is  $\frac{f'''(x)}{6}h^2$, and thus of order $h^2$ as we wanted to show. Even in a point with non-zero curvature, the error due to the curvature $f''(x)$ vanishes, and the residual error is due to the third derivative, $f'''(x)$. For a smooth function, usually $|f'''(x)| << |f''(x)|$, so the central difference scheme is expected to be very reliable, and this is why SciPy and NumPy implement this method.

Strictly speaking, the forward derivative is a good approximation for the derivative at $x+h\,/\,2$ and the backward derivative at $x-h\,/\,2$.

The central difference method may work well for smooth functions, but for data containing *noise*, it may lead to errors. In some cases, it's preferable to first interpolate the data (e.g., with a polynomial or other smooth function) and then differentiate the polynomial fit rather than the noisy data itself. An alternative is to use a smoothing filter on the data.

Lastly, more accurate methods can be obtained by forming the finite difference in a way that cancels more orders in the approximation error. An example is the Richardson extrapolation method, which is a systematic approach to decrease the error. Landau's book discusses it briefly as a method with error of order $h^4$.

## 5. Second and higher-order derivatives

So far we have discussed only first order derivatives. Higher order derivatives follow a similar approach. Second derivatives are particularly important in physics $-$ for example, Newton's law of classical mechanics, $F = m a$, requires the second order derivative of the position, the acceleration $a = \frac{d^2 x(t)}{d t^2}$.

A common way of computing second derivatives is by extending the central difference formula. The second derivative can be formally defined as

$$ 
\begin{align}
f''(x) &= \lim_{h \to 0}\, \frac{f'(x+h/2) - f'(x-h/2)}{h} \\
       &= \lim_{h \to 0}\, \frac{1}{h} \left[ \frac{f(x+h) - f(x)}{h} - \frac{f(x) - f(x-h)}{h} \right] \\
       &= \lim_{h \to 0}\, \frac{1}{h^2} \left[\,\, f(x+h) + f(x-h) - 2 f(x) \,\right]
\end{align}       
$$


We define the second derivative using central differences by removing the limit:

$$ \frac{d^2 f(x)}{dx^2}\biggr\rvert_{\rm cd} = \frac{f'_{\rm cd}(x+h/2) - f'_{\rm cd}(x-h/2)}{h} = \frac{1}{h^2} \left[\,\, f(x+h) + f(x-h) - 2 f(x) \,\right] = \frac{d^2 f(x)}{dx^2} + \mathcal{O}(h^2) $$

Once again, the truncation error is of order $h^2$, as it can be verified by Taylor expanding $f(x + h)$ and $f(x -h)$.<br>
The central difference method is thus second-order accurate for second derivatives.

### Example - Second derivative

We implement the second derivative and compute it for $f(x) = x^3$ at $x=1$ (the exact value we expect is 6).

In [ ]:
def second_derivative(f,a,h=0.01):
    return (f(a+h) + f(a-h) - 2*f(a))/h**2
    
f = lambda x: x**3
d2 = second_derivative(f,1.0,h=0.01)
print (d2)

### Second derivative with SciPy

In [ ]:
from scipy.misc import derivative
f = lambda x: x**3 
d2 = derivative(f, 1.0, dx=1e-2, n=2, args=(),order=3) #n=2: 2nd derivative
print (d2)